In [1]:
import os
import torch
gpus= [0, 1] # Rank 0 is for MI300x single device finetune, and Rank 0/1 for full 
os.environ.setdefault("CUDA_VISIBLE_DEVICES", ','.join(map(str, gpus)))
# Ensure PyTorch detects the GPUs correctly
print(f"PyTorch detected number of available devices: {torch.cuda.device_count()}") 

PyTorch detected number of available devices: 1


In [13]:
# Install PyTorch, torchvision, torchao nightlies
!pip install --pre --upgrade torch torchvision torchao --index-url https://download.pytorch.org/whl/nightly/rocm6.3/
!pip install --pre --upgrade torchtune --extra-index-url https://download.pytorch.org/whl/nightly/rocm6.3/ 
# This note book is verified under torch==2.7.0.dev20250302+rocm6.3, torchao==0.10.0.dev20250303+rocm6.3,  torchvision==0.22.0.dev20250302+rocm6.3, torchtune==0.6.0.dev20250302+rocm6.3


Looking in indexes: https://download.pytorch.org/whl/nightly/rocm6.3/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 GB 27.3 MB/s eta 0:00:0000:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 GB 59.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.7/258.7 MB 49.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.9 MB/s eta 0:00:00
  Attempting uninstall: torch━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [pytorch-triton-rocm]
    Found existing installation: torch 2.3.1+rocm5.7━━━━━━━━━━ 1/4 [pytorch-triton-rocm]
    Uninstalling torch-2.3.1+rocm5.7:╺━━━━━━━━━━━━━━━━━━━ 2/4 [torch]iton-rocm]
      Successfully uninstalled torch-2.3.1+rocm5.7━━━━━━━━━━━━━━━━ 2/4 [torch]
  Attempting uninstall: torchvision0m╺━━━━━━━━━━━━━━━━━━━ 2/4 [torch]
    Found existing installation: torchvision 0.18.1+rocm5.7━━━━━━━ 3/4 [torchvision]
    Uninstalling t

In [39]:
!tune help

usage: tune [-h] {download,ls,cp,run,validate,cat} ...
tune: error: argument {download,ls,cp,run,validate,cat}: invalid choice: 'help' (choose from download, ls, cp, run, validate, cat)


In [40]:
!tune download Qwen/Qwen3-4B  \
    --output-dir /tmp/hf_models/Qwen3-4B  \
    --ignore-patterns "original/consolidated.00.pth"

Ignoring files matching the following patterns: original/consolidated.00.pth
.gitattributes: 1.57kB [00:00, 10.7MB/s]
README.md: 16.9kB [00:00, 73.0MB/s]
generation_config.json: 100%|██████████████████| 239/239 [00:00<00:00, 4.56MB/s]
merges.txt: 1.67MB [00:00, 139MB/s]
model-00001-of-00003.safetensors: 100%|████| 3.96G/3.96G [00:02<00:00, 1.38GB/s]
model-00002-of-00003.safetensors: 100%|████| 3.99G/3.99G [00:02<00:00, 1.35GB/s]
model-00003-of-00003.safetensors: 100%|█████| 99.6M/99.6M [00:00<00:00, 172MB/s]
model.safetensors.index.json: 32.8kB [00:00, 179MB/s]
tokenizer.json: 100%|██████████████████████| 11.4M/11.4M [00:00<00:00, 47.5MB/s]
tokenizer_config.json: 9.73kB [00:00, 61.1MB/s]
vocab.json: 2.78MB [00:00, 161MB/s]
Successfully downloaded model repo and wrote to the following locations:
/tmp/hf_models/Qwen3-4B/README.md
/tmp/hf_models/Qwen3-4B/generation_config.json
/tmp/hf_models/Qwen3-4B/.gitattributes
/tmp/hf_models/Qwen3-4B/model-00003-of-00003.safetensors
/tmp/hf_models/Qw

In [74]:
!tune run lora_fine_tune_single_device --config /jupyter-tutorial/AAIPL_134_199_198_15/agen.yaml

Traceback (most recent call last):
  File "/usr/local/bin/tune", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torchtune/_cli/tune.py", line 52, in main
    parser.run(args)
  File "/usr/local/lib/python3.12/dist-packages/torchtune/_cli/tune.py", line 46, in run
    args.func(args)
  File "/usr/local/lib/python3.12/dist-packages/torchtune/_cli/run.py", line 214, in _run_cmd
    self._run_single_device(args, is_builtin=is_builtin)
  File "/usr/local/lib/python3.12/dist-packages/torchtune/_cli/run.py", line 111, in _run_single_device
    runpy.run_module(str(args.recipe), run_name="__main__")
  File "<frozen runpy>", line 222, in run_module
  File "<frozen runpy>", line 142, in _get_module_details
ImportError: No module named lora_fine_tune_single_device


In [42]:
!tune train \
  --base-model /tmp/hf_models/Qwen3-4B \
  --data-path mlabonne/guanaco-llama2-1k \
  --output-dir ./checkpoints/qwen3_finetuned \
  --learning-rate 5e-5 \
  --batch-size 2 \
  --epochs 1


usage: tune [-h] {download,ls,cp,run,validate,cat} ...
tune: error: argument {download,ls,cp,run,validate,cat}: invalid choice: 'train' (choose from download, ls, cp, run, validate, cat)


In [75]:
!tune run lora_finetune_single_device --config ../../hf_models/Qwen3-4B/config.yaml


Running LoRAFinetuneRecipeSingleDevice with resolved config:

base_model: /jupyter-tutorial/hf_models/Qwen3-4B
batch_size: 2
cutoff_len: 2048
data_path: mlabonne/guanaco-llama2-1k
device: cuda
group_by_length: false
learning_rate: 5.0e-05
lora_alpha: 16
lora_dropout: 0.05
lora_r: 8
lora_target_modules:
- q_proj
- k_proj
- v_proj
- o_proj
micro_batch_size: 2
num_epochs: 1
output_dir: ./checkpoints/qwen3_lora
save_steps: 50
tokenizer_path: /jupyter-tutorial/hf_models/Qwen3-4B
train_on_inputs: false
val_set_size: 0.1
wandb_project: qwen-lora

Traceback (most recent call last):
  File "/usr/local/bin/tune", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torchtune/_cli/tune.py", line 52, in main
    parser.run(args)
  File "/usr/local/lib/python3.12/dist-packages/torchtune/_cli/tune.py", line 46, in run
    args.func(args)
  File "/usr/local/lib/python3.12/dist-packages/torchtune/_cli/run.py", line 214, in _run_cmd
    self._run_

In [11]:
!tune lora_finetune_single_device --config qwen/4B_lora_single_device


Usage: tune [OPTIONS] COMMAND [ARGS]...
Try 'tune --help' for help.

Error: No such command 'lora_finetune_single_device'.


In [65]:
!cd ..

In [68]:
!ls ../../hf_models/Qwen3-4B

README.md		merges.txt			  tokenizer.json
Untitled.ipynb		model-00001-of-00003.safetensors  tokenizer_config.json
config.json		model-00002-of-00003.safetensors  vocab.json
config.yaml		model-00003-of-00003.safetensors
generation_config.json	model.safetensors.index.json
